In [ ]:
# Implementation Of Regression

import pandas as pd
import torch 
import numpy as np
import tensorflow as tf

In [ ]:
iono = pd.read_csv("./ionosphere_csv.csv")
iono.head()

In [3]:
X = torch.tensor(iono.drop("class", axis=1).values, dtype=torch.float)
Y = torch.tensor(
    [0 if vty == "a" else 1 if vty == "b" else 2 if vty == "c" else 3 if vty == "d" else 4 if vty == "e" else 5 if vty == "f" else 6 for vty in iono["class"]], 
    dtype=torch.long
)

print(X.shape,Y.shape)

torch.Size([351, 34]) torch.Size([351])


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

In [5]:
samples, features = X_train.shape
print(features)

34


In [ ]:
X_train = torch.tensor(X_train,dtype=float)
X_test = torch.tensor(X_test,dtype=float)
Y_train = torch.tensor(Y_train,dtype=float)
Y_test = torch.tensor(Y_test,dtype=float)

In [7]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

torch.Size([280, 34]) torch.Size([71, 34]) torch.Size([280]) torch.Size([71])


In [8]:
from torch import nn

class regression(nn.Module):
    def __init__(self):
        super(regression, self).__init__()
        self.first_layer = nn.Linear(features,34)
        self.second_layer = nn.Linear(34, 15)
        self.final_layer = nn.Linear(15,1)
        self.relu = nn.ReLU()

    def forward(self,X_batch):
        layer_out = self.relu(self.first_layer(X_batch))
        layer_out = self.relu(self.second_layer(layer_out))
        return self.final_layer(layer_out)

In [9]:
X_train = X_train.to(torch.float32)
X_train.dtype

torch.float32

In [10]:
regressor = regression()
preds = regressor(X_train[:5])
print(preds)

tensor([[-0.0032],
        [-0.0777],
        [-0.1087],
        [-0.1122],
        [-0.1199]], grad_fn=<AddmmBackward0>)


In [11]:
def TrainModel(model, loss_func, optimizer, X, Y, epochs=1000):
    for i in range(epochs):
        preds = model(X)
        preds = preds.to(torch.float)
        loss = loss_func(preds.ravel(), Y)
        loss = loss.to(torch.float)
        optimizer.zero_grad()
        # loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("MSE : {:.2f}".format(loss))

In [12]:
from torch.optim import SGD

e = 1000
learning_rate = torch.tensor(1/1e3)
learning_rate = learning_rate.to(torch.float)

regressor = regression()
mse_loss = nn.MSELoss()
optimizer = SGD(params=regressor.parameters(), lr=learning_rate)

X_train = X_train.to(torch.float)
Y_train = Y_train.to(torch.float)

In [13]:
TrainModel(regressor, mse_loss, optimizer, X_train, Y_train, epochs = e)

MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07
MSE : 24.07


In [14]:
X_test = X_test.to(torch.float)

In [15]:
train_preds = regressor(X_train)
train_preds[:5]

tensor([[-0.0324],
        [-0.0468],
        [-0.0422],
        [-0.0554],
        [-0.0188]], grad_fn=<SliceBackward0>)

In [16]:
test_preds = regressor(X_test) 
test_preds[:5]

tensor([[-0.0467],
        [-0.0339],
        [-0.0152],
        [-0.0929],
        [-0.0391]], grad_fn=<SliceBackward0>)

In [17]:
from sklearn.metrics import r2_score

print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy().squeeze(), Y_train.detach().numpy())))

Train R^2 Score : -19571.92
